In [1]:
import os
%pwd

'd:\\Project\\Machine Learning\\Text Summarization\\research'

In [2]:
os.chdir("../")
%pwd

'd:\\Project\\Machine Learning\\Text Summarization'

Entity

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvalutionConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

Configuration Manager

In [4]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_dirs

class ConfiguratioManager:
    def __init__(
            self,
            config_filepath = Config_File_Path,
            params_filepath = Params_File_Path
                ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_dirs([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvalutionConfig:
        config = self.config.model_evaluation

        create_dirs([config.root_dir])

        model_evaluation_config = ModelEvalutionConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_path = config. model_path,
            tokenizer_path = config.tokenizer_path,
            metric_file_name = config.metric_file_name
        )

        return model_evaluation_config

Component

In [5]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk, load_metric
import torch
import pandas as pd
from tqdm import tqdm

class ModelEvalution:
    def __init__(self, config: ModelEvalutionConfig):
        self.config = config
    
    def generate_batch_by_chunks(self, list_of_elements, batch_size):
        """
        split dataset into smaller batch that can be process simultaneously
        Yield successive batch-size chunks from list_of_element
        """
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]

    def calculate_metric_on_test_dataset(self, dataset, metric, model, tokenizer, batch_size=16, 
                                         device="cuda" if torch.cuda.is_available() else "cpu", 
                                         column_text ="article", column_summary= "highlights"):
        article_batches = list(self.generate_batch_by_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_by_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)
        ):
            inputs = tokenizer(article_batch, max_length=1024, truncation=True, padding="max_length", return_tensors="pt")
            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                                       attention_mask=input["attention_mask"],
                                       Length_penalty=0.8, num_beams=0, max_length=128)
            # Parameter length of penalty ensure that model doesn't generate sequences too long

            # Finally, we decode generared text
            # replace token and decode text with reference to metric

            decoded_summaries = [tokenizer.decode(s, skip_special_token=True, clean_up_tokenization_spaces=True)
                                 for s in summaries]
            decoded_summaries = [d.replace(""," ") for d in decoded_summaries]

            metric.add_batch(predictions=decoded_summaries, references=target_batch)

            # Finally compute and return ROUGE source
            score = metric.compute()
            return score
        
    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)

        #loading data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        rouge_names = ["rouge1", "rouge2", "rougeL", "rougesum"]
        rough_metric = load_metric('rouge')
        score = self.calculate_metric_on_test_dataset(
            dataset_samsum_pt['test'][0:10], rough_metric, model_pegasus, tokenizer, batch_size=2, column_text= 'dialogues', column_summary='summary'
        )

        rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
        df = pd.DataFrame(rouge_dict, index=['pegasus'])
        df.to_csv(self.config.metric_file_name, index=False)

c:\Users\DoomedMean\anaconda3\envs\textSummarize\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-07-10 03:42:04,733: INFO: config: PyTorch version 2.3.1 available.]


In [6]:
try:
    config = ConfiguratioManager()
    model_evaluation_config = config.get_model_evaluation_config
    model_evaluation = ModelEvalution(config=model_evaluation_config)
    model_evaluation.evaluate()
except Exception as e:
    raise e

[2024-07-10 03:42:05,963: INFO: common: yaml file: config\config.yaml Load Successfully]
[2024-07-10 03:42:05,966: INFO: common: yaml file: params.yaml Load Successfully]
[2024-07-10 03:42:05,968: INFO: common: Create directory at artifacts]


AttributeError: 'function' object has no attribute 'model_path'